In [ ]:
!pip install -qq transformers genomic-benchmarks datasets

     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 362 kB 78.3 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
     |████████████████████████████████| 6.6 MB 75.0 MB/s 
     |████████████████████████████████| 596 kB 69.3 MB/s 
     |████████████████████████████████| 2.3 MB 64.0 MB/s 
     |████████████████████████████████| 271 kB 72.0 MB/s 
     |████████████████████████████████| 1.1 MB 53.2 MB/s 
     |████████████████████████████████| 140 kB 68.0 MB/s 
     |████████████████████████████████| 212 kB 82.0 MB/s 
     |████████████████████████████████| 127 kB 71.6 MB/s 
     |████████████████████████████████| 94 kB 4.8 MB/s 
     |████████████████████████████████| 144 kB 68.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Parameters
MODEL_NAME = "simecek/DNADebertaK6" #Original DNABert model
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 6
STRIDE = 1

# if less than 1, only this fraction of each dataset is used
DATASET_THINING = 1

BENCHMARKS_FOLDER = '/root/.genomic_benchmarks'
# BENCHMARKS_FOLDER = '/home/jovyan/.genomic_benchmarks/' (for INFRA HUB)

DATASETS = [('demo_coding_vs_intergenomic_seqs', 0),
 ('demo_human_or_worm', 0), ('human_enhancers_cohn', 0), ('human_nontata_promoters', 0)]

BATCH_SIZE = 16
ACCUMULATION = 4
LEARNING_RATE = 1e-5
EPOCHS = 4
RUNS = 5

# do not forget to attach drive
OUTPUT_PATH = 'drive/MyDrive/genomic_benchmarks/RandomizedDNADebertaK6_2.csv'

## Download benchmark datasets and tokenizer

In [ ]:
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check.info import is_downloaded
from pathlib import Path
from tqdm.autonotebook import tqdm

for dataset_name, dataset_version in tqdm(DATASETS):
    if not is_downloaded(dataset_name):
        download_dataset(dataset_name, version=dataset_version, use_cloud_cache=True)

benchmark_root = Path(BENCHMARKS_FOLDER)

/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

In [ ]:
def kmers_strideK(s, k=K):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

if (STRIDE == 1):
  kmers = kmers_stride1
else:
  kmers = kmers_strideK

# function used for the actual tokenization
def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])))

# example
tok_func({'seq': 'ATGGAAAGAGGCACCATTCT'})    

{'input_ids': [2, 501, 1989, 3848, 3089, 56, 212, 835, 3325, 999, 3983, 3629, 2214, 650, 2587, 2142, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Looping through datasets, fine-tuning the model for each of them, logging metrics

In [ ]:
import pandas as pd
import numpy as np
from random import random, randrange
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric
from random import random, randrange

def compute_metrics_binary(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def compute_metrics_multi(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

outputs = []

for dataset_name, dataset_version in tqdm(DATASETS):
    

    labels = sorted([x.stem for x in (benchmark_root / dataset_name / 'train').iterdir()])

    tmp_dict = {}

    for split in ['train', 'test']:
        for nlabel, label in enumerate(labels):
            for f in (benchmark_root / dataset_name / split / label).glob('*.txt'):
                txt = f.read_text()
                if not DATASET_THINING or DATASET_THINING==1:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)
                elif random() < DATASET_THINING:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)

    df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    ds = Dataset.from_pandas(df)

    tok_ds = ds.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds = tok_ds.rename_columns({'cat':'labels'})

    dds = DatasetDict({
        'train': tok_ds.filter(lambda x: x["dset"] == "train").remove_columns('dset'),
        'test':  tok_ds.filter(lambda x: x["dset"] == "test").remove_columns('dset')
    })

    compute_metrics = compute_metrics_binary if len(labels) == 2 else compute_metrics_multi

    for _ in range(RUNS):

        model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))
        model_cls.deberta.encoder.layer.apply(model_cls.deberta._init_weights)

        args = TrainingArguments('outputs', learning_rate=LEARNING_RATE, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
            evaluation_strategy="epoch", per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2, gradient_accumulation_steps=ACCUMULATION,
            num_train_epochs=EPOCHS, weight_decay=0.01, save_steps=100000, seed=randrange(1,10001), report_to='none')
        
        trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                          tokenizer=tokenizer, compute_metrics=compute_metrics)
        trainer.train()
        
        max_accuracy = max([x['eval_accuracy'] for x in trainer.state.log_history if 'eval_accuracy' in x])
        max_f1 = max([x['eval_f1'] for x in trainer.state.log_history if 'eval_f1' in x]) if len(labels) == 2 else np.nan
        train_runtime = max([x['train_runtime'] for x in trainer.state.log_history if 'train_runtime' in x])
        
        outputs.append((dataset_name, max_accuracy, max_f1, train_runtime))
        outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
        outputs_df.to_csv(OUTPUT_PATH, index=False)



  0%|          | 0/4 [00:00<?, ?it/s]

Parameter 'function'=<function tok_func at 0x7f5069bd7710> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/100000 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

...

***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/simecek/DNADebertaK6/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/290389faf61feca163161b441f1533d162d3196a609b88b234318767c64960f8.c7f87ccb839059561ef6996c9f8559644ce254107c992b74bd0b46fbe8c80d51
Model config DebertaConfig {
  "_name_or_path": "simecek/DNADebertaK6",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.399097,0.827872,0.830923
1,0.477700,0.354514,0.850454,0.853487
2,0.357800,0.349729,0.851229,0.863857
3,0.316200,0.330999,0.861966,0.868502


***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/simecek/DNADebertaK6/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/290389faf61feca163161b441f1533d162d3196a609b88b234318767c64960f8.c7f87ccb839059561ef6996c9f8559644ce254107c992b74bd0b46fbe8c80d51
Model config DebertaConfig {
  "_name_or_path": "simecek/DNADebertaK6",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.404153,0.831415,0.834403
1,0.482000,0.350406,0.850343,0.853458
2,0.361900,0.329773,0.860638,0.868264
3,0.316200,0.328968,0.862077,0.870019


***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




## Outputs

In [ ]:
outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
outputs_df

,dataset,accuracy,f1,train_runtime
0,demo_coding_vs_intergenomic_seqs,0.904800,0.905175,921.0052
1,demo_coding_vs_intergenomic_seqs,0.905480,0.904997,921.1745
2,demo_coding_vs_intergenomic_seqs,0.903600,0.904029,923.4556
3,demo_coding_vs_intergenomic_seqs,0.905120,0.904922,927.2819
4,demo_coding_vs_intergenomic_seqs,0.904680,0.905978,933.9786
5,demo_human_or_worm,0.958600,0.958475,940.3017
6,demo_human_or_worm,0.958120,0.958265,934.8047
7,demo_human_or_worm,0.958240,0.958273,937.6455
8,demo_human_or_worm,0.958160,0.958213,936.3577
9,demo_human_or_worm,0.958440,0.958432,937.9097


In [ ]:
outputs_df.groupby('dataset').agg({'accuracy' : ['mean', 'sem'], 'f1' : ['mean','sem'], 'train_runtime': ['mean', 'sem']})

accuracy                  f1            \
                                      mean       sem      mean       sem   
dataset                                                                    
demo_coding_vs_intergenomic_seqs  0.904736  0.000316  0.905020  0.000311   
demo_human_or_worm                0.958312  0.000091  0.958332  0.000051   
human_enhancers_cohn              0.733650  0.000709  0.740223  0.002654   
human_nontata_promoters           0.862940  0.000464  0.869415  0.000486   

                                 train_runtime            
                                          mean       sem  
dataset                                                   
demo_coding_vs_intergenomic_seqs     925.37916  2.429831  
demo_human_or_worm                   937.40386  0.909995  
human_enhancers_cohn                 703.68634  1.006746  
human_nontata_promoters              341.54068  1.519215

In [ ]:
# saving outputs to csv file
outputs_df.to_csv(OUTPUT_PATH, index=False)